## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2023xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
import os, sys
os.path
sys.path.insert(0, 'C:\\Users\\phili\\github\\ntv-pandas')

In [3]:
from datetime import datetime
import json
from observation.cdataset import Cdataset
import pandas as pd
import ntv_pandas

def analyse_integrite(data, fields, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le dict 'fields'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement ('indic') affiché. 
    La liste des lignes en erreur est optionnellement ajoutée ('affiche') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(fields)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [4]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
#    'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    #'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  57868


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [5]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 0,5 % des points de charge sont hors itinerance

In [6]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  229
nombre de pdc en itinerance   :  57639


In [7]:
# arborescence des champs et nombre de valeurs différentes
print(Cdataset(itinerance_init).tree())

-1: root-derived (57639)
   0 : index (0 - 57639)
   1 : contact_operateur (57504 - 135)
   2 : nom_enseigne (53843 - 3796)
   3 : coordonneesXY (40131 - 17508)
   4 : adresse_station (40372 - 17267)
   5 : id_station_itineranc (32488 - 25151)
   6 : nom_station (39828 - 17811)
   7 : implantation_station (57634 - 5)
   8 : nbre_pdc (57586 - 53)
   9 : condition_acces (57637 - 2)
   10: horaires (57123 - 516)
   11: station_deux_roues (57634 - 5)
   12: id_pdc_itinerance (1650 - 55989)
   13: date_maj (56987 - 652)
   14: last_modified (57192 - 447)


-----------------------
## 4 - Bilan initial intégrité
- 15 % des lignes présentent un défaut d'intégrité

In [8]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                           3298
contact_operateur - id_station_itinerance            619
nom_enseigne - id_station_itinerance                 886
coordonneesXY - id_station_itinerance               1426
id_station_itinerance - id_pdc_itinerance           2320
nom_station - id_station_itinerance                  913
implantation_station - id_station_itinerance         399
nbre_pdc - id_station_itinerance                    2523
condition_acces - id_station_itinerance               26
horaires - id_station_itinerance                      36
station_deux_roues - id_station_itinerance            40
adresse_station - coordonneesXY                     2678

nombre d'enregistrements sans erreurs :  48819
nombre d'enregistrements avec au moins une erreur :  8820
taux d'erreur :  15  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 20% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 30% le nombre de lignes erronnées

In [9]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  1650  soit  19  %
nombre de pdc sans doublon   :  7170


In [10]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             45
nom_enseigne - id_station_itinerance                 304
coordonneesXY - id_station_itinerance                976
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  617
implantation_station - id_station_itinerance         350
nbre_pdc - id_station_itinerance                    2472
condition_acces - id_station_itinerance               14
horaires - id_station_itinerance                      24
station_deux_roues - id_station_itinerance            31
adresse_station - coordonneesXY                     2248

nombre d'enregistrements sans erreurs :  1604
nombre d'enregistrements avec au moins une erreur :  5566


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 20% le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 38 opérateurs et sont liées à des causes multiples

In [11]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['etape3_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  463  soit  8  %
nombre de pdc sans doublon   :  5103


In [12]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                 261
coordonneesXY - id_station_itinerance                439
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  336
implantation_station - id_station_itinerance         120
nbre_pdc - id_station_itinerance                    2203
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      20
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                     2203

nombre d'enregistrements sans erreurs :  107
nombre d'enregistrements avec au moins une erreur :  4996  soit  90  %


In [13]:
# structure des données résiduelles et nombre de valeurs
print(Cdataset(itinerance_5).tree())

-1: root-derived (4996)
   0 : index (0 - 4996)
   1 : contact_operateur (4958 - 38)
   2 : nom_enseigne (4587 - 409)
      9 : condition_acces (407 - 2)
   3 : coordonneesXY (3957 - 1039)
   4 : adresse_station (3558 - 1438)
   5 : id_station_itineranc (2929 - 2067)
      11: station_deux_roues (2062 - 5)
      13: date_maj (1991 - 76)
   6 : nom_station (3546 - 1450)
      14: last_modified (1392 - 58)
   7 : implantation_station (4991 - 5)
   8 : nbre_pdc (4974 - 22)
   10: horaires (4958 - 38)
   12: id_pdc_itinerance (0 - 4996)


In [14]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  7


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [15]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  4996
total des doublons à supprimer :  2113
nombre de pdc avec controles ok :  50530


In [16]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [17]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [18]:
# structure des données bonnes
print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())

-1: root-derived (50530)
   4 : id_station_itineranc (27602 - 22928)
      1 : nom_enseigne (19521 - 3407)
      2 : coordonneesXY (6809 - 16119)
         3 : adresse_station (559 - 15560)
         8 : condition_acces (16117 - 2)
      5 : nom_station (6845 - 16083)
         0 : contact_operateur (15963 - 120)
      6 : implantation_station (22923 - 5)
      7 : nbre_pdc (22875 - 53)
      9 : horaires (22439 - 489)
      10: station_deux_roues (22923 - 5)
      19: paiement_autre (22921 - 7)
      24: last_modified (22555 - 373)
   11: id_pdc_itinerance (0 - 50530)
   12: puissance_nominale (50433 - 97)
   13: prise_type_ef (50522 - 8)
   14: prise_type_2 (50522 - 8)
   15: prise_type_combo_ccs (50523 - 7)
   16: prise_type_chademo (50523 - 7)
   17: prise_type_autre (50524 - 6)
   18: paiement_acte (50522 - 8)
   20: reservation (50523 - 7)
   21: accessibilite_pmr (50526 - 4)
   22: restriction_gabarit (50436 - 94)
   23: date_maj (49901 - 629)


### Indicateurs

In [19]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [20]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  76  % des écarts

hello@powerdot.fr 1573
support@alizecharge.fr 858
sav@izivia.com 729
info@ionity.eu 324
assistance-commerciale@e-vadea.fr 321


In [21]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           3592
72  %


In [22]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 9 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [23]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3311,41047,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200011,2023-07-26,2023-10-31T07:02:06.808000+00:00
3312,41048,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200012,2023-07-26,2023-10-31T07:02:06.808000+00:00
5054,56739,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550401,2023-07-26,2023-10-31T07:02:06.808000+00:00
5055,56740,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550402,2023-07-26,2023-10-31T07:02:06.808000+00:00
5056,56741,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2023-07-26,2023-10-31T07:02:06.808000+00:00
5057,56742,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2023-07-26,2023-10-31T07:02:06.808000+00:00
5058,56743,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550405,2023-07-26,2023-10-31T07:02:06.808000+00:00


In [24]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FRS22P22220001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3311,41047,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200011,2023-07-26,2023-10-31T07:02:06.808000+00:00
3312,41048,support@alizecharge.fr,Ouest Charge,"[-3.252257, 48.239723]",2 Rue de la Gare 22110 PLOUGUERNEVEL,FRS22P22220001,Plouguernevel-Le bourg - route de Rostrenen,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200012,2023-07-26,2023-10-31T07:02:06.808000+00:00
3313,41049,support@alizecharge.fr,Ouest Charge,"[-3.313195, 48.242261]",0 ZA de Goasnel 22110 ROSTRENEN,FRS22P22220001,Rostrenen-ZA de Goasnel,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200013,2023-07-26,2023-10-31T07:02:06.808000+00:00
3314,41050,support@alizecharge.fr,Ouest Charge,"[-3.313195, 48.242261]",0 ZA de Goasnel 22110 ROSTRENEN,FRS22P22220001,Rostrenen-ZA de Goasnel,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E222200014,2023-07-26,2023-10-31T07:02:06.808000+00:00


### Cohérence implantation_station - id_station
- 120 pdc sont liés à une erreur de choix d'implantation

In [25]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,80,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2023-10-31T06:59:21.318000+00:00
19,81,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2023-10-31T06:59:21.318000+00:00
20,82,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2023-10-31T06:59:21.318000+00:00
21,83,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2023-10-31T06:59:21.318000+00:00
370,4968,support@alizecharge.fr,Métropole Rouen Normandie,"[0.872578, 49.479934]",Avenue du Président Coty 76480 DUCLAIR,FRC01P76222001,DUCLAIR - Avenue du Président Coty,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRC01E762220011,2023-07-26,2023-10-31T07:02:06.808000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5054,56739,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550401,2023-07-26,2023-10-31T07:02:06.808000+00:00
5055,56740,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550402,2023-07-26,2023-10-31T07:02:06.808000+00:00
5056,56741,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2023-07-26,2023-10-31T07:02:06.808000+00:00
5057,56742,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2023-07-26,2023-10-31T07:02:06.808000+00:00


### Cohérence nom_station - id_station
- 336 pdc sont associés à une station avec un nom non cohérent

In [26]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
23,88,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643461,2022-11-03,2023-10-31T06:59:21.318000+00:00
24,89,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643471,2022-11-03,2023-10-31T06:59:21.318000+00:00
25,90,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684251,2022-11-03,2023-10-31T06:59:21.318000+00:00
26,91,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684261,2022-11-03,2023-10-31T06:59:21.318000+00:00
27,92,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.768425, 43.958425]",PARKING ELEPHANT BLEU - LES ANGLES,FR55CP30133,PARKING ELEPHANT BLEU - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334396228947525161,2022-11-03,2023-10-31T06:59:21.318000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5054,56739,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550401,2023-07-26,2023-10-31T07:02:06.808000+00:00
5055,56740,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550402,2023-07-26,2023-10-31T07:02:06.808000+00:00
5056,56741,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2023-07-26,2023-10-31T07:02:06.808000+00:00
5057,56742,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2023-07-26,2023-10-31T07:02:06.808000+00:00


### Cohérence adresse - coordonnées
- 2203 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [27]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
107,314,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.475489, 48.824539]",PLACE VERDUN - JOINVILLE,FR55CP94340,PLACE VERDUN - JOINVILLE,Voirie,6,Accès réservé,24/7,FALSE,FR55CE943404882018224738711,2022-11-03,2023-10-31T06:59:21.318000+00:00
108,315,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.475489, 48.824539]",PLACE VERDUN - JOINVILLE,FR55CP94340,PLACE VERDUN - JOINVILLE,Voirie,6,Accès réservé,24/7,FALSE,FR55CE943404882018224738721,2022-11-03,2023-10-31T06:59:21.318000+00:00
109,316,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.475489, 48.824539]",PLACE VERDUN - JOINVILLE,FR55CP94340,PLACE VERDUN - JOINVILLE,Voirie,6,Accès réservé,24/7,FALSE,FR55CE943404882018224738731,2022-11-03,2023-10-31T06:59:21.318000+00:00
110,317,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.475489, 48.824539]",PLACE VERDUN - JOINVILLE,FR55CP94340,PLACE VERDUN - JOINVILLE,Voirie,6,Accès réservé,24/7,FALSE,FR55CE943404882018224738741,2022-11-03,2023-10-31T06:59:21.318000+00:00
111,318,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.475489, 48.824539]",PLACE VERDUN - JOINVILLE,FR55CP94340,PLACE VERDUN - JOINVILLE,Voirie,6,Accès réservé,24/7,FALSE,FR55CE943404882018224738751,2022-11-03,2023-10-31T06:59:21.318000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5098,57689,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2023-10-31T07:03:16.936000+00:00
5099,57698,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-10-31T07:03:16.445000+00:00
5100,57699,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-10-31T07:03:16.445000+00:00
5101,57700,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-10-31T07:03:16.445000+00:00


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [28]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4921,56333,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-10-31T06:59:33.891000+00:00
4922,56334,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-10-31T06:59:33.891000+00:00
4923,56335,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-10-31T06:59:33.891000+00:00
4924,56336,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-10-31T06:59:33.891000+00:00


- exemple : Stations Ouest charge

In [29]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-3.577855, 48.771084]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3306,41144,support@alizecharge.fr,Ouest Charge,"[-3.577855, 48.771084]",0 parking place du peillet 22560 TREBEURDEN,FRS22P22343001,Trebeurden-Place du Peillet,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E223430011,2023-07-26,2023-10-31T07:02:06.808000+00:00
3307,41145,support@alizecharge.fr,Ouest Charge,"[-3.577855, 48.771084]",0 parking place du peillet 22560 TREBEURDEN,FRS22P22343001,Trebeurden-Place du Peillet,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E223430012,2023-07-26,2023-10-31T07:02:06.808000+00:00
3490,41348,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22311,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22311,2023-04-07,2023-10-31T06:58:49.480000+00:00
3491,41349,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22312,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22312,2023-04-07,2023-10-31T06:58:49.480000+00:00


### Cohérence station - coordonnées
- 477 pdc sont associés à des stations avec plusieurs coordonnées

In [30]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
754,13338,web@freshmile.com,Freshmile,"[6.70933, 49.1065]",Place du Marché 57500 Saint-Avold,FRFR1PYKTKBS,"Saint-Avold, place du Marché",Voirie,3,Accès libre,24/7,false,FRFR1ECJQR2,2022-10-06,2023-10-31T07:04:08.998000+00:00
762,13901,web@freshmile.com,Freshmile,"[1.37968, 49.4055]",560 Avenue De Bonport 76320 Saint-Pierre-lès-E...,FRFR1PHIAAJ792SH,"Saint-Pierre-lès-Elbeuf, Burger King",Voirie,2,Accès libre,24/7,false,FRFR1EEYVE1,2023-10-15,2023-10-31T07:04:08.998000+00:00
763,13919,web@freshmile.com,Freshmile,"[1.03729903, 49.28873608]",560 Avenue De Bonport 76320 Saint-Pierre-lès-E...,FRFR1PHIAAJ792SH,"Saint-Pierre-lès-Elbeuf, Burger King",Voirie,2,Accès libre,24/7,false,FRFR1EFARP1,2023-10-15,2023-10-31T07:04:08.998000+00:00
765,14156,web@freshmile.com,Freshmile,"[2.378329, 48.681627]",21 Avenue De Joinville 91170 Viry-Châtillon,FRFR1PSBPX6OZUXS,"Viry-Châtillon, avenue du Général de Gaulle",Voirie,8,Accès libre,24/7,false,FRFR1EFUWM1,2023-10-16,2023-10-31T07:04:08.998000+00:00
768,14536,web@freshmile.com,Freshmile,"[6.70929, 49.1065]",Place du Marché 57500 Saint-Avold,FRFR1PYKTKBS,"Saint-Avold, place du Marché",Voirie,3,Accès libre,24/7,false,FRFR1EGYFJ1,2022-10-06,2023-10-31T07:04:08.998000+00:00
771,14661,web@freshmile.com,Freshmile,"[7.31188, 47.8772]",29 Rue D'ensisheim 68190 Ungersheim,FRFR1PQMLUBIPCHF,"Ungersheim, Salle des Fêtes",Voirie,4,Accès libre,24/7,false,FRFR1EHHGG1,2023-10-15,2023-10-31T07:04:08.998000+00:00
772,14662,web@freshmile.com,Freshmile,"[7.31206953, 47.87713109]",29 Rue D'ensisheim 68190 Ungersheim,FRFR1PQMLUBIPCHF,"Ungersheim, Salle des Fêtes",Voirie,4,Accès libre,24/7,false,FRFR1EHHGG2,2023-10-15,2023-10-31T07:04:08.998000+00:00
773,14663,web@freshmile.com,Freshmile,"[7.31206953, 47.87713109]",29 Rue D'ensisheim 68190 Ungersheim,FRFR1PQMLUBIPCHF,"Ungersheim, Salle des Fêtes",Voirie,4,Accès libre,24/7,false,FRFR1EHHGG3,2023-10-15,2023-10-31T07:04:08.998000+00:00
774,14664,web@freshmile.com,Freshmile,"[7.31206953, 47.87713109]",29 Rue D'ensisheim 68190 Ungersheim,FRFR1PQMLUBIPCHF,"Ungersheim, Salle des Fêtes",Voirie,4,Accès libre,24/7,false,FRFR1EHHGG4,2023-10-15,2023-10-31T07:04:08.998000+00:00
780,15037,web@freshmile.com,Freshmile,"[1.391725, 43.566937]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1EJVVE1,2022-11-09,2023-10-31T07:04:08.998000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [31]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,6,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-10-31T06:57:25.045000+00:00
1,7,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-10-31T06:57:25.045000+00:00
2,8,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2023-10-31T06:57:25.045000+00:00
3,9,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-10-31T06:57:25.045000+00:00
5,11,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-10-31T06:57:25.045000+00:00
6,12,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2023-10-31T06:57:25.045000+00:00
7,13,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-10-31T06:57:25.045000+00:00
8,14,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-10-31T06:57:25.045000+00:00
9,15,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2023-10-31T06:57:25.045000+00:00
10,16,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-10-31T06:57:25.045000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [32]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
28,99,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-10-31T06:59:21.318000+00:00
29,100,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-10-31T06:59:21.318000+00:00
30,101,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-10-31T06:59:21.318000+00:00
31,102,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-10-31T06:59:21.318000+00:00
32,103,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-10-31T06:59:21.318000+00:00
33,104,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-10-31T06:59:21.318000+00:00


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [33]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.658987, 48.542091]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4228,42266,support@alizecharge.fr,Ouest Charge,"[-4.658987, 48.542091]",Rue Auguste Caroff 29830 PLOUDALMEZEAU,FRS29P29178001,Ploudalmezeau-Rue Auguste Caroff,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E291780011,2023-07-26,2023-10-31T07:02:06.808000+00:00
4229,42267,support@alizecharge.fr,Ouest Charge,"[-4.658987, 48.542091]",Rue Auguste Caroff 29830 PLOUDALMEZEAU,FRS29P29178001,Ploudalmezeau-Rue Auguste Caroff,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E291780012,2023-07-26,2023-10-31T07:02:06.808000+00:00
4454,42500,sav@izivia.com,OUEST CHARGE - 29,"[-4.658987, 48.542091]",RUE AUGUSTE CAROFF,FRS29PSD2912011,PLOUDALMEZEAU-RUE AUGUSTE CAROFF,Voirie,2,Accès libre,24/7,false,FRS29ESD2912011,2023-04-14,2023-10-31T06:58:49.068000+00:00
4455,42501,sav@izivia.com,OUEST CHARGE - 29,"[-4.658987, 48.542091]",RUE AUGUSTE CAROFF,FRS29PSD2912012,PLOUDALMEZEAU-RUE AUGUSTE CAROFF,Voirie,2,Accès libre,24/7,false,FRS29ESD2912012,2023-04-14,2023-10-31T06:58:49.068000+00:00


In [34]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4122,42156,support@alizecharge.fr,Ouest Charge,"[-4.374469, 48.471878]",Route de Plabennec 29860 KERSAINT PLABENNEC,FRS29P29095001,Kersaint Plabennec-Route de Plabennec,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E290950011,2023-07-26,2023-10-31T07:02:06.808000+00:00
4123,42157,support@alizecharge.fr,Ouest Charge,"[-4.374469, 48.471878]",Route de Plabennec 29860 KERSAINT PLABENNEC,FRS29P29095001,Kersaint Plabennec-Route de Plabennec,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E290950012,2023-07-26,2023-10-31T07:02:06.808000+00:00
4748,42796,sav@izivia.com,OUEST CHARGE - 29,"[-4.374469, 48.471878]",ROUTE DE PLABENNEC,FRS29PSD296211,KERSAINT PLABENNEC-ROUTE DE PLABENNEC,Voirie,2,Accès libre,24/7,false,FRS29ESD296211,2023-04-14,2023-10-31T06:58:49.068000+00:00
4749,42797,sav@izivia.com,OUEST CHARGE - 29,"[-4.374469, 48.471878]",ROUTE DE PLABENNEC,FRS29PSD296212,KERSAINT PLABENNEC-ROUTE DE PLABENNEC,Voirie,2,Accès libre,24/7,false,FRS29ESD296212,2023-04-14,2023-10-31T06:58:49.068000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [35]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incohérence 'nbre_pdc'
- 25% des pdc ont un champ 'nbre_pdc' mal documenté

- exemple : les stations 'powerdot' avec un défaut ont un 'nbre_pdc' incohérent

In [36]:
print('PdC powerdot avec nbre_pdc incohérent : ',
      sum((itinerance_4['contact_operateur'] == 'hello@powerdot.fr') & ~(itinerance_4['nbre_pdc - id_station_itinerance'])))
print('PdC powerdot avec une incohérence : ', sum(itinerance_4['contact_operateur'] == 'hello@powerdot.fr'))
print('PdC avec nbre_pdc incohérent : ', sum(~(itinerance_4['nbre_pdc - id_station_itinerance'])))

PdC powerdot avec nbre_pdc incohérent :  1573
PdC powerdot avec une incohérence :  1573
PdC avec nbre_pdc incohérent :  2203


In [37]:
itinerance_4.loc[itinerance_4.id_station_itinerance	 == 'FRPD1PTRNORG', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'contact_operateur', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,contact_operateur,coordonneesXY,nbre_pdc
3048,FRPD1ETRNORGALFS22011,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",2
3049,FRPD1ETRNORGALFS22012,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",2
3050,FRPD1ETRNORGKPS200011,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",6
3051,FRPD1ETRNORGKPS200012,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",6
3052,FRPD1ETRNORGKPS200013,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",6
3053,FRPD1ETRNORGKPS200014,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",6
3054,FRPD1ETRNORGKPS200015,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",6
3055,FRPD1ETRNORGKPS200016,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",6
3056,FRPD1ETRNORGKPS50011,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",3
3057,FRPD1ETRNORGKPS50012,FRPD1PTRNORG,hello@powerdot.fr,"[4.8519098171272, 44.109562226158]",3


In [38]:
itinerance_4.loc[itinerance_4.contact_operateur	 == 'hello@powerdot.fr', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
1489,FRPD1EACCBEAALF0011,FRPD1PACCBEA,"[2.1122019, 49.4101655]",1
1490,FRPD1EACCBEAKP200011,FRPD1PACCBEA,"[2.1122019, 49.4101655]",4
1491,FRPD1EACCBEAKP200012,FRPD1PACCBEA,"[2.1122019, 49.4101655]",4
1492,FRPD1EACCBEAKP200013,FRPD1PACCBEA,"[2.1122019, 49.4101655]",4
1493,FRPD1EACCBEAKP200014,FRPD1PACCBEA,"[2.1122019, 49.4101655]",4
...,...,...,...,...
3057,FRPD1ETRNORGKPS50012,FRPD1PTRNORG,"[4.8519098171272, 44.109562226158]",3
3058,FRPD1ETRNORGKPS50013,FRPD1PTRNORG,"[4.8519098171272, 44.109562226158]",3
3059,FRPD1ETRNORGKPS50021,FRPD1PTRNORG,"[4.8519098171272, 44.109562226158]",3
3060,FRPD1ETRNORGKPS50022,FRPD1PTRNORG,"[4.8519098171272, 44.109562226158]",3


- exemple : station avec un pdc 'de regroupement' qui évite de documenter plusieurs lignes

In [39]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLUMEACACIAS11', 
               ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
29904,FRLUMEACACIAS11,FRLUMEACACIAS11,"[3.1485441,50.7582304]",4


- exemple : station avec plusieurs pdc mais l'id_station est identique à l'id_pdc

In [40]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[2.451322, 45.66523]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc


- exemple : station avec plusieurs pdc mais le champs a toujours une valeur de 1

In [41]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[-1.7548354193520386, 48.125067030488154]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc


### cohérence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [42]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PFLEURYFORETCENTRE', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
41612,41691,pascal.lhermitte@siege27.fr,SIEGE59,"[1.556744, 49.419466]",Centre du village Fleury la Forêt,FRS27PFLEURYFORETCENTRE,900017,Voirie,2,Accès libre,24/7,false,FRS27EFLEURYFORETCENTRED,2021-06-15,2023-10-31T06:59:05.856000+00:00
41613,41692,pascal.lhermitte@siege27.fr,SIEGE60,"[1.556744, 49.419466]",Centre du village Fleury la Forêt,FRS27PFLEURYFORETCENTRE,900017,Voirie,2,Accès libre,24/7,false,FRS27EFLEURYFORETCENTREG,2021-06-15,2023-10-31T06:59:05.856000+00:00


In [43]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLDLE0000125', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [44]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues

In [45]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence coordonnées - adresse
- exemple de plusieurs stations avec des coordonnées identiques mais des adresses différentes

In [46]:
itinerance_5.loc[itinerance_5.coordonneesXY == '[2.460441, 50.78763]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### données d'un opérateur
- exemple Chargeguru

In [47]:
itinerance_operateur = itinerance_5_full.loc[itinerance_5_full.contact_operateur == 'info@chargeguru.com']
#itinerance_operateur.loc[itinerance_operateur['adresse_station - coordonneesXY'] == False, relations]
itinerance_operateur.loc[itinerance_operateur['nom_station - id_station_itinerance'] == False, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [48]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRCG0E00052', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


In [49]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRCG0E00012', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
